# helper_functions.ipynb

These are backend functions to help login, query, and store Plaid API data

In [ ]:
#|default_exp helper_functions

In [1]:
#| export
import os
import uuid
import json
import datetime
import requests
import psycopg2
import pdb
import pandas as pd
from sqlalchemy import create_engine
from typing import Optional, Dict, Any, List, Tuple


In [ ]:
#|export
PLAID_COUNTRY_CODES = ['CA','US']
PLAID_PRODUCTS = ['transactions']
PLAID_CLIENT_ID= os.environ['PLAID_CLIENT_ID']
PLAID_SECRET= os.environ['PLAID_SECRET']
PLAID_ENV = os.environ['PLAID_ENV']
PLAID_BASE_URL = f'https://{PLAID_ENV}.plaid.com'

POSTGRES_DB="finances"
POSTGRES_HOST= "db"
POSTGRES_USER= os.environ['POSTGRES_USER']
POSTGRES_PASSWORD= os.environ['POSTGRES_PASSWORD']

## API Calls 

In [2]:
#|export

def plaid_post(endpoint: str, payload: Dict[str, Any]) -> Dict[str, Any]:
    """ 
    Makes a POST request to the Plaid API.
    
    Args:
        endpoint (str): The specific Plaid API endpoint (e.g., "accounts/get").
        payload (Dict[str, Any]): The JSON payload to be sent with the request.
    
    Returns:
        Dict[str, Any]: The JSON response from the API as a dictionary.
    """
    url = f"{PLAID_BASE_URL}/{endpoint}"
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, headers=headers, data=json.dumps(payload))
    return response.json()

def get_account(access_token: str) -> Dict[str, Any]:
    """
    Retrieves account details from the Plaid API.
    
    Args:
        access_token (str): The Plaid access token for the user account.
    
    Returns:
        Dict[str, Any]: The response from the Plaid API containing account details.
    """
    payload = {
        "client_id": PLAID_CLIENT_ID,
        "secret": PLAID_SECRET,
        "access_token": access_token
    }
    accounts_response = plaid_post("accounts/get", payload)
    return accounts_response

def get_account_transactions(access_token: str) -> List[Dict[str, Any]]:
    """
    Retrieves all transactions for an account from the Plaid API.
    
    Args:
        access_token (str): The Plaid access token for the user account.
    
    Returns:
        List[Dict[str, Any]]: A list of transaction dictionaries retrieved from the API.
    
    Raises:
        Exception: If an error occurs during the request to the Plaid API.
    """
    try:
        transactions = []
        payload = {
            "client_id": PLAID_CLIENT_ID,
            "secret": PLAID_SECRET,
            "access_token": access_token,
            "start_date": "2024-01-01",
            "end_date": "2024-12-31",
            "options": {
                "count": 100,  # Number of transactions per request
                "offset": 0    # Starting point for fetching transactions
            }
        }

        while True:
            transactions_response = plaid_post("transactions/get", payload)
            transactions.extend(transactions_response.get("transactions", []))
            
            # Stop when the total transactions have been retrieved
            if len(transactions) >= transactions_response.get("total_transactions", 0):
                break
            
            # Update the offset to fetch the next batch
            payload["options"]["offset"] += 100

        return transactions
    except Exception as e:
        print(f"There was an error. Please report outputs of this cell to the developer:\n{e}\nResponse: {transactions_response}")
        raise


## Dataframe Transformation

In [ ]:
#|export

def get_account_df(accounts_response: dict) -> pd.DataFrame:
    """
    Converts account information from the Plaid API response into a pandas DataFrame.
    
    Args:
        accounts_response (dict): The response from the Plaid API containing account information.
    
    Returns:
        pd.DataFrame: A DataFrame containing normalized account information.
    """
    return pd.json_normalize(accounts_response, sep='_')


def get_accounts_df(access_tokens: List[Tuple[str]]) -> pd.DataFrame:
    """
    Retrieves and merges account information for multiple access tokens into a single pandas DataFrame.
    
    Args:
        access_tokens (List[Tuple[str]]): A list of tuples containing access tokens.
    
    Returns:
        pd.DataFrame: A DataFrame containing merged account information from all provided access tokens.
    """
    accounts_df_list = []
    
    for single_access_token in access_tokens:
        # Retrieve and normalize account data for each access token
        account_data = get_account(single_access_token[0])
        account_df = get_account_df(account_data)
        print(account_df.head())  # Display the first few rows for debugging
        display(account_df)
        accounts_df_list.append(account_df)
    
    # Pause execution for debugging (can be removed in production)
    pdb.set_trace()
    
    # Merge all account DataFrames into one
    merged_accounts_df = pd.concat(accounts_df_list, ignore_index=True)
    
    # Pause execution for debugging (can be removed in production)
    pdb.set_trace()

    return merged_accounts_df


def get_transactions_df(access_tokens: List[Tuple[str]]) -> pd.DataFrame:
    """
    Retrieves and converts transaction data for multiple access tokens into a pandas DataFrame.
    
    Args:
        access_tokens (List[Tuple[str]]): A list of tuples containing access tokens.
    
    Returns:
        pd.DataFrame: A DataFrame containing normalized transaction data from all provided access tokens.
    """
    transactions_list = []
    
    for single_access_token in access_tokens:
        # Retrieve transactions for each access token
        transactions_list.extend(get_account_transactions(single_access_token[0]))
    
    # Normalize transaction data into a pandas DataFrame
    return pd.json_normalize(transactions_list)


## Database 


In [ ]:
#|export

def db_conn() -> Optional[psycopg2.extensions.connection]:
    """
    Creates and returns a connection to the PostgreSQL database.

    Returns:
        psycopg2.extensions.connection: A connection object to interact with the database.
        None: If the connection fails.
    """
    try:
        return psycopg2.connect(
            host=POSTGRES_HOST,
            database="finances",
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD
        )
    except psycopg2.OperationalError as e:
        print(f"Database connection skipped: {e}")
        return None


def db_sql(query: str) -> pd.DataFrame:
    """
    Executes a SQL query and returns the result as a pandas DataFrame.

    Args:
        query (str): The SQL query to execute.

    Returns:
        pd.DataFrame: A DataFrame containing the query results.
    """
    engine = create_engine(
        f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}/{POSTGRES_DB}"
    )
    try:
        dataframe = pd.read_sql_query(query, engine)
        return dataframe
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()
    finally:
        engine.dispose()


def get_stored_public_access_tokens() -> List[Tuple[str]]:
    """
    Retrieves distinct Plaid access tokens from the local database.

    Returns:
        List[Tuple[str]]: A list of tuples containing access tokens.
    """
    tokens = []

    try:
        db = db_conn()
        if not db:
            return []

        cur = db.cursor()
        cur.execute("SELECT DISTINCT plaid_access_token FROM accounts;")
        tokens = cur.fetchall()
        cur.close()
        db.close()
        print(f"Found {len(tokens)} accounts and their access tokens.")

        return tokens
    except Exception as e:
        print(f"There was an error in get_stored_public_access_tokens():\n{e}")
        return []


def insert_account_df(access_token: str, accounts_response: dict) -> None:
    """
    Inserts account information and the associated access token into the database.

    Args:
        access_token (str): The Plaid access token for the account.
        accounts_response (dict): The response from the Plaid API containing account information.
    """
    try:
        accounts_df = get_account_df(accounts_response['accounts'])

        db = db_conn()
        if not db:
            return

        cur = db.cursor()
        for _, account in accounts_df.iterrows():
            cur.execute("""
                INSERT INTO accounts (
                    account_id, 
                    mask,
                    name,
                    official_name,
                    persistent_account_id,
                    subtype,
                    type,
                    user_email,
                    user_phone,
                    plaid_access_token
                ) 
                VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                ON CONFLICT (account_id) 
                DO UPDATE SET
                    mask = EXCLUDED.mask,
                    name = EXCLUDED.name,
                    official_name = EXCLUDED.official_name,
                    persistent_account_id = EXCLUDED.persistent_account_id,
                    subtype = EXCLUDED.subtype,
                    type = EXCLUDED.type,
                    user_email = EXCLUDED.user_email,
                    user_phone = EXCLUDED.user_phone,
                    plaid_access_token = EXCLUDED.plaid_access_token;
            """, (
                account['account_id'],
                account['mask'],
                account['name'],
                account['official_name'],
                account['persistent_account_id'],
                account['subtype'],
                account['type'],
                email,
                phone,
                access_token,
            ))

        db.commit()
        cur.close()
        db.close()
        print(f"Successfully updated {accounts_df.shape[0]} accounts into the database.")
    except Exception as e:
        print(f"There was an error in insert_account_df():\n{e}")


def insert_transactions_df(transactions_df: pd.DataFrame) -> None:
    """
    Inserts transaction data into the database.

    Args:
        transactions_df (pd.DataFrame): A DataFrame containing transaction data.
    """
    try:
        db = db_conn()
        if not db:
            return

        cur = db.cursor()
        for _, transaction in transactions_df.iterrows():
            cur.execute("""
                INSERT INTO transactions (
                    transaction_id,
                    account_id,
                    amount,
                    authorized_date,
                    category_id,
                    date,
                    iso_currency_code,
                    logo_url,
                    merchant_entity_id,
                    merchant_name,
                    name,
                    payment_channel,
                    pending,
                    personal_finance_category_icon_url,
                    website,
                    personal_finance_category_detailed,
                    personal_finance_category
                ) 
                VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                ON CONFLICT (transaction_id) 
                DO UPDATE SET
                    account_id = EXCLUDED.account_id,
                    amount = EXCLUDED.amount,
                    authorized_date = EXCLUDED.authorized_date,
                    category_id = EXCLUDED.category_id,
                    date = EXCLUDED.date,
                    iso_currency_code = EXCLUDED.iso_currency_code,
                    logo_url = EXCLUDED.logo_url,
                    merchant_entity_id = EXCLUDED.merchant_entity_id,
                    merchant_name = EXCLUDED.merchant_name,
                    name = EXCLUDED.name,
                    payment_channel = EXCLUDED.payment_channel,
                    pending = EXCLUDED.pending,
                    personal_finance_category_icon_url = EXCLUDED.personal_finance_category_icon_url,
                    website = EXCLUDED.website,
                    personal_finance_category_detailed = EXCLUDED.personal_finance_category_detailed,
                    personal_finance_category = EXCLUDED.personal_finance_category;
            """, (
                transaction['transaction_id'],
                transaction['account_id'],
                transaction['amount'],
                transaction['authorized_date'] if pd.notnull(transaction['authorized_date']) else None,
                transaction['category_id'],
                transaction['date'],
                transaction['iso_currency_code'],
                transaction['logo_url'] if pd.notnull(transaction['logo_url']) else None,
                transaction['merchant_entity_id'] if pd.notnull(transaction['merchant_entity_id']) else None,
                transaction['merchant_name'] if pd.notnull(transaction['merchant_name']) else None,
                transaction['name'],
                transaction['payment_channel'],
                transaction['pending'],
                transaction['personal_finance_category_icon_url'] if pd.notnull(transaction['personal_finance_category_icon_url']) else None,
                transaction['website'] if pd.notnull(transaction['website']) else None,
                transaction['personal_finance_category.detailed'],
                transaction['personal_finance_category.primary']
            ))

        db.commit()
        cur.close()
        db.close()
        print(f"Successfully inserted {transactions_df.shape[0]} transactions into the database.")
    except Exception as e:
        print(f"There was an error in insert_transactions_df():\n{e}")


def upsert_account_balances_df(accounts_df: pd.DataFrame) -> None:
    """
    Inserts or updates account balance history in the database.

    Args:
        accounts_df (pd.DataFrame): A DataFrame containing account balance data.
    """
    try:
        db = db_conn()
        if not db:
            return

        cur = db.cursor()
        for _, account in accounts_df.iterrows():
            cur.execute("""
                INSERT INTO accounts_balance_history (
                    account_id, 
                    balances_available,
                    balances_current,
                    balances_iso_currency_code,
                    balances_limit,
                    balances_unofficial_currency_code,
                    balances_datetime
                ) 
                VALUES (
                    %s, %s, %s, %s, %s, %s, %s
                );
            """, (
                account['account_id'],
                account['balances_available'],
                account['balances_current'],
                account['balances_iso_currency_code'],
                account['balances_limit'],
                account['balances_unofficial_currency_code'],
                datetime.datetime.now(),
            ))

        db.commit()
        cur.close()
        db.close()
        print(f"Successfully updated {accounts_df.shape[0]} account balances into the database.")
    except Exception as e:
        print(f"There was an error in upsert_account_balances_df():\n{e}")


## Authentication

In [4]:
#|export

def generate_link_token(email: str, phone: str) -> Optional[str]:
    """
    Generates a link token to authenticate with the Plaid API.

    Args:
        email (str): The user's email address.
        phone (str): The user's phone number.

    Returns:
        Optional[str]: The generated link token if successful, otherwise None.
    """
    try:
        payload = {
            "client_id": PLAID_CLIENT_ID,
            "secret": PLAID_SECRET,
            "client_name": "Jupyter Notebook",
            "country_codes": PLAID_COUNTRY_CODES,
            "language": "en",
            "user": {
                "client_user_id": str(uuid.uuid4()),
                "phone_number": phone,
                "email_address": email
            },
            "hosted_link": {},
            "products": PLAID_PRODUCTS
        }
        link_token_response = plaid_post("link/token/create", payload)

        # Display the hosted link and expiration
        print(
            "Navigate to this page and authenticate with your bank: ",
            link_token_response.get("hosted_link_url", "No URL provided"),
            "\nThis link expires: ",
            link_token_response.get("expiration", "No expiration provided")
        )
        return link_token_response.get('link_token')

    except Exception as e:
        print(f"There was an error in generate_link_token:\n{e}")
        return None


def get_and_save_public_token(link_token: str) -> None:
    """
    Retrieves a public token using the link token and exchanges it for an access token.
    The account information is then saved to the database.

    Args:
        link_token (str): The link token generated during the Plaid Link flow.

    Returns:
        None
    """
    exchange_response = ''
    access_token = ''

    try:
        # Step 1: Retrieve link token details
        payload = {
            "client_id": PLAID_CLIENT_ID,
            "secret": PLAID_SECRET,
            "link_token": link_token
        }
        link_token_details = plaid_post("link/token/get", payload)

        # Extract the public token from the link sessions
        public_token = (
            link_token_details['link_sessions'][0]['results']['item_add_results'][0]['public_token']
        )

        # Step 2: Exchange the public token for an access token
        payload = {
            "client_id": PLAID_CLIENT_ID,
            "secret": PLAID_SECRET,
            "public_token": public_token
        }
        exchange_response = plaid_post("item/public_token/exchange", payload)
        access_token = exchange_response.get("access_token")

        if access_token:
            print("Access token has been generated successfully")
        else:
            print("Failed to generate access token.")
            return

    except KeyError as ke:
        print(f"Key error occurred in get_and_save_public_token: Missing key {ke}")
        return
    except Exception as e:
        print(f"There was an error in get_and_save_public_token:\n{e}\nResponse: {exchange_response}")
        return

    # Step 3: Fetch account information and save to the database
    try:
        accounts_response = get_account(access_token)
        insert_account_df(access_token, accounts_response)
    except Exception as e:
        print(f"There was an error while saving account information:\n{e}")


## Tasks

In [ ]:
#|export

def get_and_save_all_account_transactions() -> None:
    """
    Retrieves all account transactions for stored public access tokens
    and inserts the transactions into the database.

    Steps:
    1. Fetch public access tokens from the database.
    2. Retrieve transactions data for each account associated with the tokens.
    3. Insert the retrieved transactions into the database.

    Returns:
        None
    """
    try:
        # Step 1: Fetch public access tokens
        access_tokens = get_stored_public_access_tokens()

        # Step 2: Retrieve all transactions as a DataFrame
        transactions_df = get_transactions_df(access_tokens)

        # Step 3: Insert transactions into the database
        insert_transactions_df(transactions_df)
        print("Successfully retrieved and saved all account transactions.")

    except Exception as e:
        print(f"An error occurred in get_and_save_all_account_transactions:\n{e}")


def get_and_save_balance_history() -> None:
    """
    Retrieves account balance history for stored public access tokens
    and updates the balance history in the database.

    Steps:
    1. Fetch public access tokens from the database.
    2. Retrieve account details for each account associated with the tokens.
    3. Update the account balance history in the database.

    Returns:
        None
    """
    try:
        # Step 1: Fetch public access tokens
        access_tokens = get_stored_public_access_tokens()

        # Step 2: Retrieve all account details as a DataFrame
        accounts_df = get_accounts_df(access_tokens)

        # Step 3: Update account balance history in the database
        upsert_account_balances_df(accounts_df)
        print("Successfully retrieved and saved account balance history.")

    except Exception as e:
        print(f"An error occurred in get_and_save_balance_history:\n{e}")


In [5]:
#|export

def about():
    print("="*60)
    print("="*60)
    print(f"Jupyter Finances")
    # Date/Time
    print(f"Current Date and Time: {datetime.datetime.now()}")
    
    print("="*60)
    print(f"Plaid API ({PLAID_ENV})")
    print("="*60)
    print(f"PLAID_CLIENT_ID: {PLAID_CLIENT_ID}")
    print(f"PLAID_PRODUCTS: {PLAID_PRODUCTS}")
    print(f"PLAID_COUNTRY_CODES: {PLAID_COUNTRY_CODES}")
    print("="*60)
